Update scripts for giving tuesday monitoring and review

In [1]:
# Make sure you have the correct Python version and required libraries -- AFTON
# 1. Insatall some Python 3.9 version via .venv (Visual Studio Code will let you create this in the upper right)
# 2. Acivatate the new env in the terminal via 
#   $ .\venv\Scripts\activate or $ \venv\Scripts\Activate.ps1 (Windows)
#   or $ source venv/bin/activate (Mac)
# 3. Install requireed libraries at the terminal by running $ pip install -r requirements.txt
#   You might need $ sudo pip install -r requirements.txt instead if you have permission errors.

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd
import numpy as np
from datetime import datetime

## Update this with actual date if necessary (e.g. if doing a practice run) -- AFTON
gt_date = '2025-12-02'
day_after_date = '2025-12-03'

# 2024
gt_date_2024 = '2024-12-03'
day_after_date_2024 = '2024-12-04'

# 2023
# gt_date = '2023-11-28'
# day_after_date = '2023-11-29'

# 2023 -- Skip this whole section

In [ ]:
q = '''select
            count(distinct(org)) as orgs,
            sum(case when recurring=0 or recurring_origin=1 then amount else null end) as amount,
            count(distinct(case when recurring=0 or recurring_origin=1 then id else null end)) as count,
            max(amount) as max_amount,
            avg(case when recurring=0 or recurring_origin=1 then amount else null end) as avg_amount,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
            count(distinct(case when (source='PP' or channel !=0) then id else null end)) as digital_wallet_count
        from transactions
        where
            status='A' and
            (date='2023-11-28' or (date='2023-11-29' and hour<5))'''
redshift_query_read(q, schema='production')

In [64]:
## Run this and copy paste orgs, count, and volume into the hourly updates for Fundraising -- AFTON
q = f'''select
            date,
            hour,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume,
            sum(case when recurring=0 then amount else null end) as volume_onetime,
            max(amount) as max_amount
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or (date='{day_after_date}' and hour<=5))
        group by date, hour'''
df = redshift_query_read(q, schema='production')
df.sort_values(['date', 'hour'], ascending=True)

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,date,hour,orgs,count,volume,volume_onetime,max_amount
11,2025-12-02,0,182,231,46043.66,45590.17,10000.0
14,2025-12-02,1,135,169,24935.21,24830.21,2079.0
16,2025-12-02,2,74,81,23565.51,23254.14,7000.0
8,2025-12-02,3,51,60,10379.24,7407.99,2600.0
17,2025-12-02,4,70,77,23519.25,23457.45,10300.0
9,2025-12-02,5,135,160,28096.51,28013.31,2060.0
7,2025-12-02,6,324,475,97331.68,96965.37,5247.5
0,2025-12-02,7,635,1226,256426.57,253944.76,15000.0
10,2025-12-02,8,1198,2884,712623.46,691327.43,25000.0
4,2025-12-02,9,1553,4471,921458.28,912588.61,20000.0


In [ ]:
q = '''select
            distinct(org) as org
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (date='2023-11-28' or 
                (date='2023-11-29' and hour<5))
        '''
orgs = redshift_query_read(q, schema='production')

print("{:,} orgs".format(len(orgs)))

orgs.to_csv("gt.orgs.2023.csv", index=False)

# 2025

Update transactions table

In [ ]:
# !/home/ubuntu/anaconda3/bin/python ../../scripts/export\ transactions/export_all.py
# ! ../../.venv/bin/python ../../scripts/export\ transactions/export_all.py

## Updates

### Update hourly counts

In [82]:
## Run this and copy paste orgs, count, and volume into the hourly updates for Fundraising -- AFTON
q = f'''select
            date,
            hour,
            count(distinct(org)) as orgs,
            count(distinct(id)) as count,
            sum(amount) as volume,
            sum(case when recurring=0 then amount else null end) as volume_onetime,
            max(amount) as max_amount
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or (date='{day_after_date}' and hour<=5))
        group by date, hour'''
df = redshift_query_read(q, schema='production')
df.sort_values(['date', 'hour'], ascending=True)

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,date,hour,orgs,count,volume,volume_onetime,max_amount
12,2025-12-02,0,182,231,46043.66,45590.17,10000.0
4,2025-12-02,1,135,169,24935.21,24830.21,2079.0
11,2025-12-02,2,74,81,23565.51,23254.14,7000.0
14,2025-12-02,3,51,60,10379.24,7407.99,2600.0
0,2025-12-02,4,70,77,23519.25,23457.45,10300.0
15,2025-12-02,5,135,160,28096.51,28013.31,2060.0
18,2025-12-02,6,324,475,97331.68,96965.37,5247.5
2,2025-12-02,7,635,1226,256426.57,253944.76,15000.0
16,2025-12-02,8,1198,2884,712623.46,691327.43,25000.0
25,2025-12-02,9,1553,4471,921458.28,912588.61,20000.0


### Update daily totals

#### covers datapoints for the entire day

In [19]:
## Run this and put the recurring count and the digital wallet count in the final tab for fundraising -- AFTON
q = f'''select
            count(distinct(org)) as orgs,
            sum(case when recurring=0 or recurring_origin=1 then amount else null end) as amount,
            count(distinct(case when recurring=0 or recurring_origin=1 then id else null end)) as count,
            max(amount) as max_amount,
            avg(case when recurring=0 or recurring_origin=1 then amount else null end) as avg_amount,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
            count(distinct(case when (source='PP' or channel !=0) then id else null end)) as digital_wallet_count,
            sum(case when recurring=0 then amount else null end) as amount_onetime,
            sum(case when recurring!=0 then amount else null end) as amount_rec,
            sum(case when recurring_origin!=0 then amount else null end) as amount_rec_orig
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            ((date='{gt_date}') or (date='{day_after_date}' and hour<=5))
        '''
df = redshift_query_read(q, schema='production')
df

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,orgs,amount,count,max_amount,avg_amount,recurring_count,digital_wallet_count,amount_onetime,amount_rec,amount_rec_orig
0,5191,17284854.24,83158,72800.0,207.855579,1774,5967,17133935.05,150919.19,150919.19


### bucketed totals for new story -- one-off request

In [20]:
# get bucketed amounts for news story
# <= 5 is changed to <5 because technically that goes to 6am not 5am. It didn't matter on GT because we pulled the numbers at 5am but 
# in a retrospective it would matter, and would pull extra data
q = f'''select
            case 
                when amount <1 then '0. <$1'
                when amount >= 1 and amount <= 50 then '1. $1-50'
                when amount > 50 and amount <= 100 then '2. $50.01-100'
                when amount > 100 and amount <= 500 then '3. $100.01-500' 
                when amount > 500 and amount <= 1000 then '4. $500.01-1000'
                when amount > 1000 and amount <= 5000 then '5. $1000.01-5000'
                when amount > 5000 and amount <= 10000 then '6. $5000.01-10000' 
                when amount >10000 then '7. >$10000' 
                end as amountRange
            , count(distinct(id)) as count
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            ((date='{gt_date}') or (date='{day_after_date}' and hour<5))
        group by amountRange
        order by amountRange
        '''
df = redshift_query_read(q, schema='production')
df

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,amountrange,count
0,0. <$1,2820
1,1. $1-50,26658
2,2. $50.01-100,20061
3,3. $100.01-500,26841
4,4. $500.01-1000,3878
5,5. $1000.01-5000,2554
6,6. $5000.01-10000,178
7,7. >$10000,56


#### representative forms data points

In [ ]:
# q = '''select
#             org,
#             form,
#             sum(case when recurring=0 or recurring_origin=1 then amount else null end) as amount,
#             count(distinct(case when recurring=0 or recurring_origin=1 then id else null end)) as count,
#             max(amount) as max_amount,
#             avg(case when recurring=0 or recurring_origin=1 then amount else null end) as avg_amount,
#             count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count,
#             count(distinct(case when (source='PP' or channel !=0) then id else null end)) as digital_wallet_count,
#             sum(case when recurring=0 then amount else null end) as amount_onetime,
#             sum(case when recurring!=0 then amount else null end) as amount_rec,
#             sum(case when recurring_origin!=0 then amount else null end) as amount_rec_orig
#         from transactions
#         where
#             (status='A' or status='P') and
#             (recurring=0 or recurring_origin=1) and
#             (date='2024-12-03' or (date='2024-12-04' and hour<=5))
#         group by form, org'''
# df_forms = redshift_query_read(q, schema='production')

In [ ]:
# rep_forms = pd.read_csv("../representative forms/filtered_forms.csv")['form'].unique()

# df_forms['is_rep'] = df_forms['form'].astype(int).isin([int(f) for f in rep_forms])

In [ ]:
# print(len(rep_forms))

# df_forms['is_rep'].value_counts()

In [ ]:
# print("2024, representative forms vs the rest")

# cols = ['amount', 'count', 'max_amount', 'avg_amount', 
#         'recurring_count', 'digital_wallet_count', 
#         'amount_onetime', 'amount_rec', 'amount_rec_orig']
# df_forms.groupby('is_rep')[cols].agg(['mean', 'median']).reset_index()

In [ ]:
# print("2024 representative forms YoY")



#### digital wallets

In [4]:
dp_2023 = 820 / 35072
dp_2024 = 2665 / 54919

print("Digital Wallets participation:")
print("-"*40)
print("Qgiv:")
print("{:.2f}% of 2023 transactions".format(dp_2023 * 100.))
print("{} ({:.2f}%) 2024 orgs".format(114, (114 / 1789) * 100.))
print("{:.2f}% of 2024 transactions".format(dp_2024 * 100.))
print("{} ({:.2f}%) 2024 orgs".format(418, (418 / 3227) * 100.))

dp_b_2023 = 4060 / 80534
dp_b_2024 = 9295 / 90973
print()
print("Bloom:")
print("{:.2f}% of 2023 transactions".format(dp_b_2023 * 100.))
print("{} ({:.2f}%) 2023 orgs".format(114, (1060 / 5832) * 100.))
print("{:.2f}% of 2024 transactions".format(dp_b_2024 * 100.))
print("{} ({:.2f}%) 2024 orgs".format(418, (1513 / 6286) * 100.))

Digital Wallets participation:
----------------------------------------
Qgiv:
2.34% of 2023 transactions
114 (6.37%) 2024 orgs
4.85% of 2024 transactions
418 (12.95%) 2024 orgs

Bloom:
5.04% of 2023 transactions
114 (18.18%) 2023 orgs
10.22% of 2024 transactions
418 (24.07%) 2024 orgs


In [5]:
q = '''select
            count(distinct(org)) as orgs,
            count(distinct(case when (source='PP' or channel !=0) then org else null end)) as digital_wallet_org
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='2023-11-28' or (date='2023-11-29' and hour<5))
        '''
dworgs = redshift_query_read(q, schema='production')
dworgs

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,orgs,digital_wallet_org
0,1789,114


In [6]:
orgs_2023_bl = 5832
orgs_2024_bl = 6286

vol_2023_bl = 29094088.25
vol_2024_bl = 32215240.97

print("Bloom:")
print("2023: ${:,.2f}/org".format(vol_2023_bl / orgs_2023_bl))
print("2024: ${:,.2f}/org".format(vol_2024_bl / orgs_2024_bl))

orgs_2023 = 1793
orgs_2024 = 3227

vol_2023 = 6784094.73
vol_2024 = 11180830.48

print()
print("Qgiv:")

print("2023: ${:,.2f}/org".format(vol_2023 / orgs_2023))
print("2024: ${:,.2f}/org".format(vol_2024 / orgs_2024))

print()
print("Combined:")

print("2023: ${:,.2f}/org".format((vol_2023 + vol_2023_bl) / (orgs_2023 + orgs_2023_bl)))
print("2024: ${:,.2f}/org".format((vol_2024 + vol_2024_bl) / (orgs_2024 + orgs_2024_bl)))

Bloom:
2023: $4,988.70/org
2024: $5,124.92/org

Qgiv:
2023: $3,783.66/org
2024: $3,464.78/org

Combined:
2023: $4,705.34/org
2024: $4,561.77/org


In [ ]:
# print("Qgiv forms:")
# digital_wallet_forms = df_forms[df_forms['digital_wallet_count']>0]['form'].unique()
# perc_dw = len(digital_wallet_forms) / len(df_forms)
# print("{:,} ({:.2f}%) forms used digital wallets".format(len(digital_wallet_forms), perc_dw * 100.))

# df_forms['dw_ratio'] = df_forms['digital_wallet_count'] / df_forms['count']
# mn = df_forms[df_forms['digital_wallet_count']>0]['dw_ratio'].mean() * 100.
# mdn = df_forms[df_forms['digital_wallet_count']>0]['dw_ratio'].median() * 100.
# print("among forms using digital wallet, {:.2f}% mean, {:.2f}% median of transactions were digital wallet transactions".format(mn, mdn))

### Update segment counts

In [86]:
## Run this and put count and sum in the NTEE tab -- AFTON

q = f'''select
            t.org as org,
            o.segment as segment,
            t.id as id,
            t.amount as amount
        from transactions as t
            left join organization as o on t.org=o.id
        where
            t.status='A' and
            (t.date='{gt_date}' or (t.date='{day_after_date}' and t.hour<5))
        '''
df = redshift_query_read(q, schema='production')
segs = df.groupby('segment').agg({'id': 'nunique',
                                  'amount': 'mean'}).reset_index()

segs['median'] = df.groupby('segment')['amount'].median().reset_index()['amount']
segs['sum'] = df.groupby('segment')['amount'].sum().reset_index()['amount']
segs['count'] = segs['id']
segs['mean'] = segs['amount']

segs.drop(['id', 'amount'], axis=1, inplace=True)

segs[['segment', 'count', 'sum', 'mean', 'median']]

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,segment,count,sum,mean,median
0,A - Arts; Culture; and Humanities,953,125855.46,132.062392,52.500
1,B - Educational Institutions,2655,636213.54,239.628452,100.000
2,C - Environmental Advocacy and Protection,1412,177913.01,126.000715,51.500
3,D - Animal,6086,704128.10,115.696369,51.750
4,E - Health; General and Rehabilitative,1111,247711.77,222.962889,75.000
5,F - Mental Health and Crisis Intervention,673,128706.46,191.242883,78.750
6,G - Disease; Disorders; Medical Disciplines,1482,238929.55,161.221019,53.000
7,H - Medical Research,566,155530.53,274.788922,100.000
8,I - Crime; Legal Related,260,79477.89,305.684192,103.000
9,J - Employment; Job Related,89,24340.64,273.490337,100.000


### Export org ID's for Jaime

In [85]:
## Run this and slack the file gt.orgs.2025.cvs to Jaime -- AFTON (ideally run this second, after the first query to check for data)
q = f'''select
            distinct(org) as org
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or 
                (date='{day_after_date}' and hour<=5))
        '''

orgs = redshift_query_read(q, schema='production')
orgs['time'] = datetime.now()

print("{:,} orgs".format(len(orgs)))

orgs.to_csv("gt.orgs.2025.csv", index=False)

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


5,191 orgs


In [ ]:
## Run this and slack the file gt.orgs.2024.cvs to Jaime -- AFTON (ideally run this second, after the first query to check for data)

# temp query to mimic 2024
q = f'''select
            distinct(org) as org
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or 
                (date='{day_after_date}' and hour<=5))
        '''

orgs = redshift_query_read(q, schema='production')
orgs['time'] = '2024-12-03 23:59'

print("{:,} orgs".format(len(orgs)))

orgs.to_csv("gt.orgs.2024.all.csv", index=False)

In [ ]:
q = '''select
            date,
            hour,
            org
        from transactions
        where
            (status='A' or status='P') and
            (recurring=0 or recurring_origin=1) and
            (date='2024-12-03' or 
                (date='2024-12-04' and hour<=5))
        '''
orgs = redshift_query_read(q, schema='production').drop_duplicates()

print("{:,} org hourly entries".format(len(orgs)))

orgs.to_csv("gt.orgs-hourly.2024.csv", index=False)

In [ ]:
orgs.head()

In [ ]:
sorted(orgs['hour'].unique())

## Followups after GT day

In [12]:
## Avg total fundraising rev per NP
q = f'''select
            org,
            extract(year from date) as year,
            sum(amount) as totalAmount,
            count(distinct id) as transactionCount 
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or 
                (date='{day_after_date}' and hour<=5)
                or date = '{gt_date_2024}' or
                (date ='{day_after_date_2024}' and hour<=5))
        group by org, extract(year from date)
        '''
df = redshift_query_read(q, schema='production')

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
df.head()

,org,year,totalamount,transactioncount
0,444931,2024,8551.00,104
1,451129,2024,3304.00,30
2,446162,2024,70.00,2
3,445637,2024,51.50,1
4,452889,2024,9467.13,51


In [14]:
df.groupby('year').totalamount.mean()

year
2024    3444.619836
2025    3320.992536
Name: totalamount, dtype: float64

In [ ]:
df.groupby('year').transactioncount.mean()

year
2024    16.998451
2025    16.011370
Name: transactioncount, dtype: float64

In [ ]:
## Avg recurring rev per NP -- recurring donations only
q = f'''select
            org,
            extract(year from date) as year,
            count(distinct(case when recurring_origin=1 then id else null end)) as recurring_count
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (date='{gt_date}' or 
                (date='{day_after_date}' and hour<=5)
                or date = '{gt_date_2024}' or
                (date ='{day_after_date_2024}' and hour<=5))
        group by org, extract(year from date)
        '''
df = redshift_query_read(q, schema='production')

In [19]:
df.groupby('year').recurring_count.mean()

year
2024    0.345212
2025    0.340913
Name: recurring_count, dtype: float64

In [25]:
## Avg digital wallet rev per NP 
q = f'''select
            org,
            extract(year from date) as year,
            sum(amount) as amount,
            count(distinct id) as transaction_count
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (source='PP' or channel !=0) and
            (date='{gt_date}' or 
                (date='{day_after_date}' and hour<=5)
                or date = '{gt_date_2024}' or
                (date ='{day_after_date_2024}' and hour<=5))
        group by org, extract(year from date)
        '''
df = redshift_query_read(q, schema='production')

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
df.head()

,org,year,amount,transaction_count
0,453325,2024,1819.75,3
1,452366,2024,3368.25,34
2,454122,2025,101.50,2
3,447141,2025,846.40,11
4,456284,2025,3254.00,34


In [27]:
df.groupby('year').amount.mean()

year
2024    916.259689
2025    906.215662
Name: amount, dtype: float64

In [29]:
df.groupby('year').amount.max()

year
2024    26629.17
2025    29362.95
Name: amount, dtype: float64

In [28]:
df.groupby('year').transaction_count.sum()

year
2024    2665
2025    5967
Name: transaction_count, dtype: int64

# scratch

In [ ]:
q = '''select
            id,
            date,
            hour,
            org,
            form,
            amount,
            recurring,
            recurring_origin
        from transactions
        where
            status='A' and
            (recurring=0 or recurring_origin=1) and
            (date='2024-12-03' or (date='2024-12-04' and hour<3))
        '''
df = redshift_query_read(q, schema='production')

In [ ]:
print("{:,} rows; {:,} unique IDs".format(len(df), len(df['id'].unique())))

In [ ]:
mid = df[df['hour']==0]

In [ ]:
mid.sort_values('amount', ascending=False).head(15)

In [ ]:
vol_2024_qgiv = 5733960.85
vol_2023_qgiv = 3437545.60

count_2024_qgiv = 27426
count_2023_qgiv = 17301

vol_2024_blm = 35716583.59
vol_2023_blm = 24716618.74

count_2024_blm = 61585
count_2023_blm = 54211

In [ ]:
count_2023 = count_2023_qgiv + count_2023_blm
vol_2023 = vol_2023_qgiv + vol_2023_blm

count_2024 = count_2024_qgiv + count_2024_blm
vol_2024 = vol_2024_qgiv + vol_2024_blm

perc_diff_count = (count_2024 - count_2023) / count_2023
perc_diff_vol = (vol_2024 - vol_2023) / vol_2023

print("2023 both:")
print("\tcount: {:,}".format(count_2023))
print("\tvolume: ${:,.2f}".format(vol_2023))
print("2024 both:")
print("\tcount: {:,} ({:.2f}%)".format(count_2024, perc_diff_count * 100.))
print("\tvolume: ${:,.2f} ({:.2f}%)".format(vol_2024, perc_diff_vol * 100.))
print()

perc_count = (count_2024_qgiv - count_2023_qgiv) / count_2023_qgiv
perc_vol = (vol_2024_qgiv - vol_2023_qgiv) / vol_2023_qgiv

print("Qgiv YoY:")
print("\tcount: {:.2f}%".format(perc_count * 100.))
print("\tvol: {:.2f}%".format(perc_vol * 100.))